In [1]:
import pandas as pd
import numpy as np

In [2]:
#User features 
user_features = pd.read_csv("csvfiles/user_features_train.csv")


In [3]:
#Users meta data 
users = pd.read_csv("csvfiles/users_train.csv")

In [4]:
#Target_Train 
target = pd.read_csv("csvfiles/targets_train.csv")

In [5]:
user_features.head()

,ID,first_prediction,RetentionD0,RetentionD1,RetentionD2,RetentionD3,RetentionD4,RetentionD5,RetentionD6,RetentionD7,...,IAPRevenueD6,IAPRevenueD7,IAPRevenueD8,IAPRevenueD9,IAPRevenueD10,IAPRevenueD11,IAPRevenueD12,IAPRevenueD13,IAPRevenueD14,IAPRevenueD15
0,0,3.314099,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1.681524,True,False,False,False,False,False,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,10.718750,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,5.100000,True,True,True,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2.091409,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
users.head()

,ID,first_open_date,first_open_timestamp,local_first_open_timestamp,country,platform,device_category,device_brand,device_model,has_ios_att_permission,ad_network
0,0,2024-03-02,1709355895042000,1709334295042000,Mexico,Android,mobile,Xiaomi,Redmi A2,False,unityads_int
1,1,2024-03-19,1710824539731000,1710806539731000,Peru,Android,mobile,Samsung,Galaxy A13,False,applovin_int
2,2,2024-03-18,1710731043082000,1710720243082000,Brazil,Android,mobile,Xiaomi,Redmi 12,False,applovin_int
3,3,2024-03-03,1709455862260000,1709441462260000,Dominican Republic,iOS,mobile,Apple,iPhone 11 Pro Max,False,NaN
4,4,2024-04-30,1714482477190000,1714464477190000,Ecuador,Android,mobile,Motorola,Moto E22,False,applovin_int


In [7]:
target.head()

,ID,TARGET
0,0,0.000000
1,1,0.018892
2,2,0.000000
3,3,0.046650
4,4,0.014680


In [8]:
user_features.isna().sum()

ID                      0
first_prediction    25735
RetentionD0             0
RetentionD1             0
RetentionD2             0
                    ...  
IAPRevenueD11           0
IAPRevenueD12           0
IAPRevenueD13           0
IAPRevenueD14           0
IAPRevenueD15           0
Length: 76, dtype: int64

In [9]:
target.isna().sum()

ID        0
TARGET    0
dtype: int64

In [10]:
users.isna().sum()

ID                                 0
first_open_date                    0
first_open_timestamp               0
local_first_open_timestamp         0
country                           82
platform                           0
device_category                    0
device_brand                    5840
device_model                       0
has_ios_att_permission             0
ad_network                    310470
dtype: int64

In [11]:
users["has_ios_att_permission"].value_counts()

has_ios_att_permission
False    764921
True     113673
Name: count, dtype: int64

In [12]:
users["device_brand"].value_counts()

device_brand
Apple          363623
Samsung        225831
Xiaomi          82158
Motorola        76009
OPPO            22920
                ...  
Aiprotablet         1
Fossibot            1
Meitu               1
Tagital             1
Movisun             1
Name: count, Length: 295, dtype: int64

In [13]:
users["ID"]

0              0
1              1
2              2
3              3
4              4
           ...  
878589    878589
878590    878590
878591    878591
878592    878592
878593    878593
Name: ID, Length: 878594, dtype: int64

In [14]:
merged_df = pd.merge(users, user_features, on = "ID")
df = pd.merge(merged_df, target, on="ID")

In [15]:
#Leaving outside the players that uninstalled the game

retention_columns = [f"RetentionD{i}" for i in range(1, 16)]

#A function to calculate last active day of player

def last_active_day(row):
    for day in range(15,0,-1):
        if row[f"RetentionD{day}"] == True:
            return day
    return 0 


In [16]:
df["Last Active Day"] = df.apply(last_active_day, axis=1)

In [17]:
df["Last Active Day"].head(10)

0    12
1     9
2     0
3     2
4     0
5     2
6     2
7     0
8     6
9     1
Name: Last Active Day, dtype: int64

In [18]:
def count_retention_values(row):
    true_count = sum(row[col] == True for col in retention_columns)
    false_count = sum(row[col] == False for col in retention_columns)
    return pd.Series([true_count, false_count], index=['True_count', 'False_count'])

In [19]:
#8. günden sonra oynaması lazım 5 günden az oynamaması lazım

df[['True_count', 'False_count']] = df.apply(count_retention_values, axis=1)